# ResNet
微软亚洲研究院的研究员设计了更深但结构更加简单的网络 ResNet，并且凭借这个网络子在 2015 年 ImageNet 比赛上大获全胜。

ResNet 有效地解决了深度神经网络难以训练的问题，可以训练高达 1000 层的卷积网络。网络之所以难以训练，是因为存在着梯度消失的问题，离 loss 函数越远的层，在反向传播的时候，梯度越小，就越难以更新，随着层数的增加，这个现象越严重。ResNet 通过引入了跨层链接解决了梯度回传消失的问题。

In [1]:
import tensorflow as tf
from utils import cifar10_input

In [2]:
batch_size = 64
train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(image_size=96, batch_size=batch_size)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Q

In [3]:
import tensorflow.contrib.slim as slim

In [4]:
def subsample(x, factor, scope=None):
    if factor == 1:
        return x
    return slim.max_pool2d(x, [1,1], factor, scope=scope)

In [5]:
def residual_block(x, bottleneck_depth, out_depth, stride=1, scope='residual_block'):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope):
        if in_depth == out_depth:
            shortcut = subsample(x, stride, 'shortcut')
        
        else:
            shortcut = slim.conv2d(x, out_depth, [1, 1], stride=stride, activation_fn=None, scope='shortcut')
        
        residual = slim.conv2d(x, bottleneck_depth, [1, 1], stride=1, scope='conv1')
        residual = slim.conv2d(residual, bottleneck_depth, 3, stride, scope='conv2')
        residual = slim.conv2d(residual, out_depth, [1, 1], stride=1, activation_fn=None, scope='conv3')
        
        output = tf.nn.relu(shortcut + residual)
        
        return output

In [6]:
def ResNet(inputs, num_classes, reuse=None, is_training=None, verbose=False):
    with tf.variable_scope('resnet', reuse=reuse):
        net = inputs
    
        if verbose:
            print('input:{}'.format(net.shape))
        
        with slim.arg_scope([slim.batch_norm], is_training=is_training):
            with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], padding='SAME'):
                with tf.variable_scope('block1'):
                    net = slim.conv2d(net, 32, [5, 5], stride=2, scope='conv5_5')

                    if verbose:
                        print('block1:{}'.format(net.shape))

                with tf.variable_scope('block2'):
                    net = slim.max_pool2d(net, [3,3], 2, scope='max_pool')
                    net = residual_block(net, 32, 128, scope='residual_block1')
                    net = residual_block(net, 32, 128, scope='residual_block2')

                    if verbose:
                        print('block2:{}'.format(net.shape))

                with tf.variable_scope('block3'):
                    net = residual_block(net, 128, 512, stride=2, scope='residual_block1')
                    net = residual_block(net, 128, 512, scope='residual_block2')

                    if verbose:
                        print('block3:{}'.format(net.shape))

                with tf.variable_scope('classificatiom'):
                    net = tf.reduce_mean(net, [1,2], name='global_pool', keep_dims=True)
                    net = slim.flatten(net, scope='flatten')
                    net = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None, scope='logit')

                    if verbose:
                        print('classification:{}'.format(net.shape))

                return net

In [7]:
with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, normalizer_fn=slim.batch_norm) as sc:
    conv_scope = sc

In [8]:
is_training = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(conv_scope):
    train_out = ResNet(train_imgs, 10, is_training=is_training, verbose=True)
    val_out = ResNet(val_imgs, 10, is_training=is_training, reuse=True)

input:(64, 96, 96, 3)
block1:(64, 48, 48, 32)
block2:(64, 24, 24, 128)
block3:(64, 12, 12, 512)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
classification:(64, 10)


In [9]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [10]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

In [ ]:
from utils.learning import train_with_bn
train_with_bn(train_op, train_loss, train_acc, val_loss, val_acc, 20000, is_training)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
[train]: step 0 loss = 2.3063 acc = 0.1250 (0.0187 / batch)
[val]: step 0 loss = 2.3051 acc = 0.1094
